# Imports

In [8]:
import pandas as pd
import copy

In [10]:
df_raw = pd.read_csv("~/code/moth_project/0_database/tracking/track_localize_classify_annotation-2022_05_13.csv")
#df_raw.head(30)
df = df_raw.copy()
df_raw.head(3)

,image,track_id,bb_topleft_x,bb_topleft_y,bb_botright_x,bb_botright_y,bb_centre_x,bb_centre_y,class,subclass,confidence
0,20220513203542-00-01.jpg,NaN,3422,958,3550,1106,3486,1032,nonmoth,nonmoth,95
1,20220513203542-00-01.jpg,NaN,620,971,744,1110,682,1040,nonmoth,nonmoth,94
2,20220513203542-00-01.jpg,NaN,2977,805,3095,933,3036,869,nonmoth,nonmoth,94


In [ ]:
def trackIDS(df):
    pass
def searching_for_same_species():
    pass
def calculate_threshold():
    pass
def modify_IDs():
    pass
def apply_IDs_modification():
    searching_for_same_species()
    calculate_threshold()
    modify_IDs()
    pass

def main():
    '''
    TODO: 
    1- Extract the track IDs of moths. 
    2- For each track ID, search on the image near by if another bbox with the same label and a good confidence score is here. Then calculate the cost function.
    3- If cost function is below the threshold, apply the modification of the track ID on each image.
    '''
    ids = trackIDS(df)
    apply_IDs_modification()
    
    pass

main()